In [1]:
import os
import sys
from pathlib import Path

# Add parent directory to Python path for imports
# This ensures we can import from _utils regardless of where the notebook is run from
current_dir = Path().resolve()
# If we're in the basics directory, go up one level; otherwise use current directory
if current_dir.name == 'basics':
    parent_dir = current_dir.parent
else:
    parent_dir = current_dir

if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

from openai import OpenAI
from _utils.website_scraper import fetch_website_contents
from _utils.standard_functions import load_env
from IPython.display import Markdown
from IPython.display import display, update_display


In [3]:
# reading the env
load_env()

api_key = os.getenv('OPENAI_API_KEY')
base_url = os.getenv("OPENAI_BASE_URL")

OpenAI API Key exists and begins with sk-proj-
Google API Key exists and begins with AI
Anthropic API Key not set
DeepSeek API Key not set
Groq API Key not set
Grok API Key not set
OpenRouter API Key not set


In [4]:
# setting up the usual objects
openai = OpenAI(base_url=base_url, api_key=api_key)
model = "gpt-5-nano"
website_to_review="https://bramptonbjj.com"

To summarize any website, we need to break down our problem into the following sub-steps:

1. Scan all the available links in the website map - using a python library                <Standard Programing>
2. Ask AI to ignore the irrelevant links and retain information about the relevant pages   <AI Intelligence>
3. AI will scan the link pages and grab the relevant content                               <AI Intelligence>
4. AI will then summarize that info and return the details                                 <AI Intelligence>

Step 1 - List all the weblinks for the supplied website

In [5]:
# custom function in the website_scrapper.py file
website_links = fetch_website_contents(website_to_review)
print(website_links)

Brazilian Jiu Jitsu - Brampton Braziian Jiu Jitsu Academy

HOME
BOOK A FREE CLASS
PRICE LIST
SCHEDULE
INSTRUCTORS
BRAZILIAN JIU JITSU
CONTACT
GEORGETOWN LOCATION
ORANGEVILLE LOCATION
More
HOME
BOOK A FREE CLASS
PRICE LIST
SCHEDULE
INSTRUCTORS
BRAZILIAN JIU JITSU
CONTACT
GEORGETOWN LOCATION
ORANGEVILLE LOCATION
HOME
BOOK A FREE CLASS
PRICE LIST
SCHEDULE
INSTRUCTORS
BRAZILIAN JIU JITSU
CONTACT
GEORGETOWN LOCATION
ORANGEVILLE LOCATION
Brampton Brazilian Jiu Jitsu Academy Inc.
Brampton Brazilian Jiu Jitsu Academy Inc.
Brampton Brazilian Jiu Jitsu Academy Inc.
Brampton Brazilian Jiu Jitsu Academy Inc.
Fostering positive lifestyles through Brazilian Jiu Jitsu
Brampton Brazilian Jiu Jitsu Academy Inc.
Brampton Brazilian Jiu Jitsu Academy Inc.
Brampton Brazilian Jiu Jitsu Academy Inc.
Brampton Brazilian Jiu Jitsu Academy Inc.
Fostering positive lifestyles through Brazilian Jiu Jitsu
Free Trial
Our Schedule
Our Classes
It is important to us that Brampton BJJ is the right fit for you before you 

We're going to ask the AI to fetch the details, by passing on instructions in the user prompt

In [6]:
user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.
"""

In [8]:
system_prompt = """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Use a combination of paragraphs and bullet points.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

Like before, we'll create the messages object and call the OpenAI object

In [9]:
def messages(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [10]:
messages(website_to_review)

[{'role': 'system',
  'content': '\nYou are a snarky assistant that analyzes the contents of a website,\nand provides a short, snarky, humorous summary, ignoring text that might be navigation related.\nUse a combination of paragraphs and bullet points.\nRespond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.\n'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or announcements, then summarize these too.\nhttps://bramptonbjj.com'}]

We will use the similar implementation to call OpenAPI as before, but we'll just wrap it in a function to make it easier for us

In [11]:
def summarize_website(url):
    website = fetch_website_contents(url)
    response = openai.chat.completions.create(model = model, messages = messages(website))
    return response.choices[0].message.content

In [12]:
summarize_website(website_to_review)

'Brampton Brazilian Jiu Jitsu Academy is basically a gym pitch for “fostering positive lifestyles through Brazilian Jiu Jitsu,” with a free-trial escape hatch to try before you commit to the spandex life.\n\nWhat this site conveys (in a nutshell):\n- Free trial first, enrollment later: semi-private intro lesson plus a loaner uniform if you’ve never rolled before.\n- Wide net of classes: for kids, adults, men, women, newbies, and pro competitors—so basically something for everyone wearing a gi.\n- Flexible schedule: early mornings, noons, and evenings to fit your chaotic life.\n- Locations: Brampton main site plus Georgetown and Orangeville branches (so you’re never too far from a mat).\n- Contact and booking: a simple online form to book the free trial, with standard privacy/recaptcha disclaimers.\n\nAnnouncements and news:\n- The site shows a dated item from 14-12-19 about booking a free trial, but there’s no visible current news update beyond that. In other words, looks like an old p

In [13]:
# Make it presentable
def summarize_website_cleanly(url):
    website = fetch_website_contents(url)
    response = openai.chat.completions.create(model = model, messages = messages(website))
    return display(Markdown(response.choices[0].message.content))

In [14]:
summarize_website_cleanly(website_to_review)

Here’s the gist of the Brampton Brazilian Jiu Jitsu Academy site, with a dash of sass.

- What it is: A Brampton BJJ academy that pitches you a free trial, multiple locations, and a habit of rolling on the mat with everyone from kids to pros.

- Free trial: Comes with a semi-private intro lesson and a free loaner uniform. Perfect if you don’t own a gi or have never tied a belt in your life.

- Classes: Broad appeal — kids, adults, men, women, beginners, and professional competitors. If you can stand on a mat, there’s probably a class for you.

- Schedule: Claimed to be diverse with early mornings, noons, and evenings to fit a busy life.

- Locations: Brampton main site (address provided), plus Georgetown and Orangeville locations listed.

- Contact and sign-up: Contact email (info@bramptonbjj.ca) and phone number (416-230-9653). There’s a booking form to get you in the door, plus a Ramadan-style flood of navigation links you’ll ignore.

- Tagline: Fostering positive lifestyles through Brazilian Jiu Jitsu — the vibes are strong, whether you’re here for sport, fitness, or self-defense.

- Miscellany: The page is full of repeated menu items and sections, which makes it feel repetitive or cluttered. There’s a line that looks like a date (14-12-19) near the booking area, which doesn’t clearly read as a real current announcement.

News or announcements:
- There isn’t a clear News or Announcements section. The only dated line (14-12-19) near the booking prompt doesn’t amount to a current update. So, basically: no fresh news, just a lot of “join us” vibes.

Overall vibe: A straightforward gym-site pitch with a strong emphasis on free trials and accessibility for all skill levels, wrapped in a slightly repetitive navigation layout.

In [16]:
# Make a fancy summary
def summarize_website_fancy(url):
    website = fetch_website_contents(url)
    stream = openai.chat.completions.create(model = model, messages = messages(website), stream=True)

    response = ""

    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [17]:
summarize_website_fancy(website_to_review)

Here’s the Brampton Brazilian Jiu Jitsu Academy page, with the sarcasm dial set to “friendly but judgey.” 

- What it is
  - A Brampton BJJ school that also mentions Georgetown and Orangeville locations. It’s all about fostering “positive lifestyles” through Brazilian Jiu Jitsu.

- What they offer
  - Classes for everyone: kids, adults, men, women, newbies, and even professional competitors. If you can walk, you can probably start taking classes.
  - A flexible schedule designed to fit busy lives: early mornings, noons, and evenings.

- Free trial details
  - Free Trial promised: includes a semi-private lesson and a free loaner uniform. Perfect for absolute beginners who want to see if BJJ is their thing without committing right away.

- Scheduling and convenience
  - They emphasize a diverse schedule and a trial that makes the first step low-friction. There’s a form to book a trial right on the site (name, email, and a “Send” button).

- Locations and contact
  - Main Brampton location: 1295 Williams Pkwy E, Brampton, ON L6S 3J8. Contact: info@bramptonbjj.ca, 416-230-9653.
  - Mentions additional locations in Georgetown and Orangeville (likely other venues or branches).

- News/announcements
  - The page doesn’t show obvious current news. The only dated element visible is a “14-12-19” line, which looks like an old post or placeholder rather than a real news update. No fresh announcements here.

- Miscellaneous vibe
  - The site repeats navigation items a lot (typical for a basic or cloned site), but the core message is simple: come try BJJ with a free trial, and they’ll see if you’re a fit.

BONUS: Let's try the same fancy outcome using local ollama

In [21]:
OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL")
ollama_model = "llama3.2"
ollama_openai_compliant_client = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

In [22]:
def summarize_website_fancy_ollama(url):
    website = fetch_website_contents(url)
    stream = ollama_openai_compliant_client.chat.completions.create(model = ollama_model, messages = messages(website), stream=True)

    response = ""

    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [23]:
summarize_website_fancy_ollama(website_to_review)

**Summary**
So, you want to get your Brazilian Jiu Jitsu (BJJ) fix in Brampton? Well, luck is on your side because this academy offers a "free trial" – aka a semi-private lesson and a free loaner uniform. Because who needs money when you can learn karate-jitsu? On the bright side, their instructors are diverse.

**News/Announcements**
• [12/14/19] Book your free trial! Included is a free loaner uniform and introduction lesson.
• They remind us that they have two locations: Georgetown and Orangeville. Maybe consider switching up your workout routine?

**What's Not Said Much of Anything**
* No classes for seniors or people with pets. 
* Pricing (there it is, hidden like the Emperor), but no links to any page